In [ ]:
%cd /content/drive/MyDrive/Home_Depot_Case_Study/Workspace_Deployment
!pwd

/content/drive/MyDrive/Home_Depot_Case_Study/Workspace_Deployment
/content/drive/MyDrive/Home_Depot_Case_Study/Workspace_Deployment


## Imports and Loadings

In [ ]:
import pandas as pd 
import numpy as np 
import regex as re
!pip install nltk 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import PorterStemmer 
from wordcloud import STOPWORDS 
from prettytable import PrettyTable
import warnings
warnings.filterwarnings('ignore')
import math
from nltk import sent_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from numpy.linalg import norm
import pickle
from tqdm.notebook import tqdm
from scipy.stats import uniform, randint, loguniform
!pip install rank_bm25

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### BM25

In [ ]:
database  = pd.read_csv('database.csv')
product_text = database['text'].values
with open('BM25_model.pkl', 'rb') as f:
  bm25_model = pickle.load(f)

#### LTOR

**CLEANING**

In [ ]:
with open('Final/cleaning/unique_brands.pkl','rb') as f:
  unique_brands = pickle.load(f)

def first_n(n, sent):
  if n > len(sent.split()):
    return 'error101'
  return ' '.join(sent.split()[:n])

def fillna_brand(data, unique_brnds):
  null_df = data[data['brand'].isnull()]
  notnull_df = data.dropna()

  for i, row in null_df.iterrows():
    title = row['product_title']
    if first_n(4, title) in unique_brnds:
      null_df['brand'].loc[i] = first_n(4, title)
    elif first_n(3, title) in unique_brnds:
      null_df['brand'].loc[i] = first_n(3, title)
    elif first_n(2, title) in unique_brnds:
      null_df['brand'].loc[i] = first_n(2, title)
    else:
      null_df['brand'].loc[i] = first_n(1, title)

  data['brand'].loc[null_df.index] = null_df['brand'].values
  return data

def fillna_attributes(data):
  null_df = data[data['combined_attr'].isnull()]
  null_df['combined_attr'] = null_df['product_description'].copy()
  data['combined_attr'].loc[null_df.index] = null_df['combined_attr'].values
  return data

In [ ]:
def standardize_units(text):
  text = " " + text + " "
  text = re.sub('( gal | gals | galon )',' gallon ',text)
  text = re.sub('( ft | fts | feets | foot | foots )',' feet ',text)
  text = re.sub('( squares | sq )',' square ',text)
  text = re.sub('( lb | lbs | pounds )',' pound ',text)
  text = re.sub('( oz | ozs | ounces | ounc )',' ounce ',text)
  text = re.sub('( yds | yd | yards )',' yard ',text)
  return text

def preprocessing(sent):
  sent = sent.replace('in.', ' inch ') #If we dont to this then 'in.' will be turned to 'in' in the next step
  words = re.split(r'\W+', sent)
  words = [word.lower() for word in words]
  res = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", ' '.join(words)) #add space between number and alphabets in a string
  cleaned = standardize_units(res) 
  cleaned = ' '.join(cleaned.split()) #removing extra whitespaces
  return cleaned

def preprocessing_search(sent):
  sent = sent.replace('in.', ' inch ')
  words = re.split(r'\W+', sent)
  words = [word.lower() for word in words]
  res = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", ' '.join(words)) #add space between number and alphabets in a string
  res = standardize_units(res) 
  res = res.replace(' in ', ' inch ') #in search_terms 'in' is used more for 'inch' than as a preposition hence this step shouldn't hurt
  cleaned = ' '.join(res.split()) #removing extra whitespaces
  return cleaned

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('Final/cleaning/corpus.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or set([word]))
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def corrected_term(term):
  temp = term.lower().split()
  temp = [correction(word) for word in temp]
  return ' '.join(temp)

In [ ]:
#stop word removal and stemming
#We didn't do this before because we wanted to fix the typos in the searh term first 
porter = PorterStemmer()
stp_wrds = set(stopwords.words('english'))

def futher_preprocessing(sent):
  sent = sent.replace('_', ' _ ')
  words = sent.split()
  words = [w for w in words if not w in stp_wrds]
  words = [porter.stem(word) for word in words]
  return ' '.join(words)

#stop word removal only - no stemming
def futher_preprocessing_without_stem(sent):
  sent = sent.replace('_', ' _ ')
  words = sent.split()
  words = [w for w in words if not w in stp_wrds]
  return ' '.join(words)

**FEATURIZATION**

In [ ]:
def common_words(df, col1, col2):
  """
  Returns common words between each row of col1 and col2 of df in the form of a list. 
  Length of list is number of rows in dataframe
  """
  common_list = []
  for i, row in df[[col1,col2]].iterrows():
    set1 = set(row[col1].split())
    set2 = set(row[col2].split())
    common = set1 & set2
    common = ' '.join(common)
    common_list.append(common)
  return common_list

def cosine_similarity_sent(sent1, sent2):
  """
  Cosine Similarity between 2 sentences treating them as sets of words
  """
  set1 = set(sent1.split())
  set2 = set(sent2.split())
  numerator = len(set1 & set2)
  denominator = math.sqrt(len(set1)) * math.sqrt(len(set2))
  if not denominator:
      return 0.0
  else:
      return numerator / denominator

def jacquard_coefficient_sent(sent1, sent2):
  """
  Jacquard Coefficient between 2 sentences treating them as sets of words
  """
  set1 = set(sent1.split())
  set2 = set(sent2.split())
  numerator = len(set1 & set2)
  denominator = len(set1 | set2)
  if not denominator:
      return 0.0
  else:
      return numerator / denominator

In [ ]:
corrected_feat_set = ['num_common_ST','num_common_SD', 'num_common_SB', 'cosine_ST', 'cosine_SD', 
                'cosine_SB', 'jacquard_ST', 'jacquard_SD', 'jacquard_SB', 
                'len_description', 'len_brand', 'len_title', 'len_search',
                'islast_ST', 'islast_SD', 'islast_SB']

raw_feat_set = ['num_common_r_ST', 'num_common_r_SD', 'num_common_r_SB', 'cosine_r_ST', 'cosine_r_SD', 
                'cosine_r_SB',  'jacquard_r_ST', 'jacquard_r_SD', 'jacquard_r_SB', 
                'len_description', 'len_brand', 'len_title', 'len_r_search', 
                'islast_r_ST','islast_r_SD', 'islast_r_SB']

feat_set1_comb =  ['num_common_ST','num_common_SD', 'num_common_SB', 'cosine_ST', 'cosine_SD', 
                'cosine_SB', 'jacquard_ST', 'jacquard_SD', 'jacquard_SB', 
                'len_description', 'len_brand', 'len_title', 'len_search',
                'islast_ST', 'islast_SD', 'islast_SB', 'num_common_r_ST', 'num_common_r_SD', 'num_common_r_SB', 'cosine_r_ST', 'cosine_r_SD', 
                'cosine_r_SB',  'jacquard_r_ST', 'jacquard_r_SD', 'jacquard_r_SB', 
                'len_r_search', 'islast_r_ST','islast_r_SD', 'islast_r_SB']

In [ ]:
def cosine_similarity_vec(a, b):
  """
  Cosine Similarity between 2 vectors
  """  
  num = np.dot(a, b)
  den = norm(a)*norm(b)
  if den != 0:
    return num/den
  else:
    return 0

def jacquard_similarity_vec(a, b):
  """
  Cosine Similarity between 2 vectors
  """  
  num = np.dot(a,b)
  den = norm(a)**2 + norm(b)**2 - np.dot(a,b)
  if den != 0:
    return num/den
  else:
    return 0

def inner_product_vec(a, b):
  return np.dot(a,b)

In [ ]:
with open('Final/featurization/F2_tfidf_search.pkl','rb') as f:
  F2_tfidf_search = pickle.load(f)
with open('Final/featurization/F2_tsvd_search.pkl','rb') as f:
  F2_tsvd_search = pickle.load(f)

with open('Final/featurization/F2_tfidf_title.pkl','rb') as f:
  F2_tfidf_title = pickle.load(f)
with open('Final/featurization/F2_tsvd_title.pkl','rb') as f:
  F2_tsvd_title = pickle.load(f)

with open('Final/featurization/F2_tfidf_desc.pkl','rb') as f:
  F2_tfidf_desc = pickle.load(f)
with open('Final/featurization/F2_tsvd_desc.pkl','rb') as f:
  F2_tsvd_desc = pickle.load(f)

with open('Final/featurization/F2_tfidf_lsi.pkl','rb') as f:
  F2_tfidf_lsi = pickle.load(f)
with open('Final/featurization/F2_tsvd_lsi.pkl','rb') as f:
  F2_tsvd_lsi = pickle.load(f)

In [ ]:
def lmir_fit(corpus):
  words = ' '.join(corpus).split()
  freq_dict = Counter(words)
  total_words = len(words)
  params = {
      'freq_dict':freq_dict,
      'total_words':total_words
  }
  return params

def lmir_jm_score(query, doc, params, lambd):
  query = query.split()
  doc = doc.split()
  if len(doc) != 0 and len(query) != 0:
    eps = 0.0001/(params['total_words'])
    score = 0
    for word in query:
      p_ml = doc.count(word) / len(doc)
      if word in params['freq_dict'].keys():
        p_c = params['freq_dict'][word] / params['total_words']
      else: 
        p_c = 0
      score += np.log(lambd*p_ml + (1-lambd)*p_c + eps)
    return score

def lmir_dir_score(query, doc, params, mu):
  query = query.split()
  doc = doc.split()
  if len(doc) != 0 and len(query) != 0:
    eps = 0.0001/(params['total_words'])
    score = 0
    for word in query:
      p_ml = doc.count(word) / len(doc)
      if word in params['freq_dict']:
        p_c = params['freq_dict'][word] / params['total_words']
      else: 
        p_c = 0
      lambd = len(doc) / (len(doc) + mu)
      score += np.log(lambd*p_ml + (1-lambd)*p_c + eps)
    return score

def lmir_abs_score(query, doc, alpha):
  query = query.split()
  doc = doc.split()
  if len(doc) != 0 and len(query) != 0:
    score = 0
    temp_dict = {k:v+alpha for k,v in Counter(doc).items()}
    for word in query:
      if word in temp_dict:
        pass
      else:
        temp_dict[word] = alpha
    denominator = sum(temp_dict.values())
    for word in query:
      score += temp_dict[word] / denominator
  return score

with open('Final/featurization/F3_LM_params_title.pkl','rb') as f:
  params_title_LM = pickle.load(f)
with open('Final/featurization/F3_LM_params_brand.pkl','rb') as f:
  params_brand_LM = pickle.load(f)
with open('Final/featurization/F3_LM_params_desc.pkl','rb') as f:
  params_desc_LM = pickle.load(f)

In [ ]:
def bm25_fit(corpus):
  tfidf_model = TfidfVectorizer(smooth_idf=False, token_pattern=r"(?u)\b\w+\b")
  tfidf_model.fit(corpus)
  idf_dict = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
  avgdl = np.mean([len(doc.split()) for doc in corpus])
  params = {'idf_dict': idf_dict, 
            'avgdl' : avgdl,
            'N' : N}
  return params

def bm25_score(query, doc, params, k=0.1, b=0.5):
  idf_dict = params['idf_dict']
  avgdl = params['avgdl']
  N = params['N']
  score_query = 0
  for word in query.split():
    dl = len(doc.split())
    tf = doc.count(word)
    if word in idf_dict.keys():
      idf = idf_dict[word]
    else: 
      idf = np.log(N+1) 
    score_word = idf*(tf*(k+1))/(tf + k*(1-b) + b*dl/avgdl)
    score_query += score_word
  return score_query

with open('Final/featurization/F3_bm25_params_title.pkl','rb') as f:
  params_title_bm25 = pickle.load(f)
with open('Final/featurization/F3_bm25_params_desc.pkl','rb') as f:
  params_desc_bm25 = pickle.load(f)
with open('Final/featurization/F3_bm25_params_brand.pkl','rb') as f:
  params_brand_bm25 = pickle.load(f)

In [ ]:
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

with open('Final/featurization/F3_tfidf_w2v_params_search.pkl','rb') as f:
  tfidf_w2v_params_search = pickle.load(f)
with open('Final/featurization/F3_tfidf_w2v_params_title.pkl','rb') as f:
  tfidf_w2v_params_title = pickle.load(f)
with open('Final/featurization/F3_tfidf_w2v_params_desc.pkl','rb') as f:
  tfidf_w2v_params_desc = pickle.load(f)

In [ ]:
with open('Final/featurization/F3_SmM_params_title.pkl','rb') as f:
  F3_SmM_params_title = pickle.load(f)
with open('Final/featurization/F3_SmM_params_brand.pkl','rb') as f:
  F3_SmM_params_brand = pickle.load(f)
with open('Final/featurization/F3_SmM_params_desc.pkl','rb') as f:
  F3_SmM_params_desc = pickle.load(f)

**MODELLING**

In [ ]:
F1_scaler_ridge = pickle.load(open('Final/modelling/base_models/F1_scaler_ridge.pkl', 'rb'))
F1_scaler_lasso = pickle.load(open('Final/modelling/base_models/F1_scaler_lasso.pkl', 'rb'))
F1_scaler_en = pickle.load(open('Final/modelling/base_models/F1_scaler_en.pkl', 'rb'))
F1_scaler_svr = pickle.load(open('Final/modelling/base_models/F1_scaler_svr.pkl', 'rb'))

F1_xgb = pickle.load(open('Final/modelling/base_models/F1_xgb.pkl', 'rb'))
F1_rf = pickle.load(open('Final/modelling/base_models/F1_rf.pkl', 'rb'))
F1_ridge = pickle.load(open('Final/modelling/base_models/F1_ridge.pkl', 'rb'))
F1_lasso = pickle.load(open('Final/modelling/base_models/F1_lasso.pkl', 'rb'))
F1_en = pickle.load(open('Final/modelling/base_models/F1_en.pkl', 'rb'))
F1_svr = pickle.load(open('Final/modelling/base_models/F1_svr.pkl', 'rb'))
F1_dt = pickle.load(open('Final/modelling/base_models/F1_dt.pkl', 'rb'))

In [ ]:
#Loading the standard scalers
F2_scaler_ridge = pickle.load(open('Final/modelling/base_models/F2_scaler_ridge.pkl', 'rb'))
F2_scaler_lasso = pickle.load(open('Final/modelling/base_models/F2_scaler_lasso.pkl', 'rb'))
F2_scaler_en = pickle.load(open('Final/modelling/base_models/F2_scaler_en.pkl', 'rb'))
F2_scaler_svr = pickle.load(open('Final/modelling/base_models/F2_scaler_svr.pkl', 'rb'))

#Loading the models 
F2_svr = pickle.load(open('Final/modelling/base_models/F2_svr.pkl', 'rb'))
F2_rf = pickle.load(open('Final/modelling/base_models/F2_rf.pkl', 'rb'))
F2_ridge = pickle.load(open('Final/modelling/base_models/F2_ridge.pkl', 'rb'))
F2_lasso = pickle.load(open('Final/modelling/base_models/F2_lasso.pkl', 'rb'))
F2_en = pickle.load(open('Final/modelling/base_models/F2_en.pkl', 'rb'))
F2_dt = pickle.load(open('Final/modelling/base_models/F2_dt.pkl', 'rb'))

In [ ]:
#Loading the standard scalers
F3_scaler_en = pickle.load(open('Final/modelling/base_models/F3_scaler_en.pkl', 'rb'))
F3_scaler_ridge = pickle.load(open('Final/modelling/base_models/F3_scaler_ridge.pkl', 'rb'))
F3_scaler_lasso = pickle.load(open('Final/modelling/base_models/F3_scaler_lasso.pkl', 'rb'))

#Loading the models 
F3_ridge = pickle.load(open('Final/modelling/base_models/F3_ridge.pkl', 'rb'))
F3_lasso = pickle.load(open('Final/modelling/base_models/F3_lasso.pkl', 'rb'))
F3_en = pickle.load(open('Final/modelling/base_models/F3_en.pkl', 'rb'))
F3_dt = pickle.load(open('Final/modelling/base_models/F3_dt.pkl', 'rb'))

In [ ]:
final_scaler = pickle.load(open('Final/modelling/meta_scaler.pkl', 'rb'))
final_ridge = pickle.load(open('Final/modelling/meta_ridge.pkl', 'rb'))

## Final Functions

In [ ]:
def get_search_relevance(test_set):
  test_set = fillna_brand(test_set, unique_brands)
  test_set = fillna_attributes(test_set)
  test_set = test_set.fillna('')

  test_set['cleaned_title'] = test_set['product_title'].apply(lambda x : preprocessing(x))
  test_set['cleaned_brand'] = test_set['brand'].apply(lambda x : preprocessing(x))
  test_set['cleaned_description'] = test_set['product_description'].apply(lambda x : preprocessing(x))
  test_set['cleaned_attributes'] = test_set['combined_attr'].apply(lambda x : preprocessing(x))
  test_set['cleaned_search'] = test_set['search_term'].apply(lambda x : preprocessing_search(x))

  test_set['corrected_search'] = test_set['cleaned_search'].apply(lambda x: corrected_term(x))

  cleaned_test_set = pd.DataFrame() 
  cleaned_test_set['title'] = test_set['cleaned_title'].apply(lambda x : futher_preprocessing(x)) 
  cleaned_test_set['brand'] = test_set['cleaned_brand'].apply(lambda x : futher_preprocessing(x)) 
  cleaned_test_set['description'] = test_set['cleaned_description'].apply(lambda x : futher_preprocessing(x)) 
  cleaned_test_set['attributes'] = test_set['cleaned_attributes'].apply(lambda x : futher_preprocessing(x)) 
  cleaned_test_set['search'] = test_set['cleaned_search'].apply(lambda x : futher_preprocessing(x)) 
  cleaned_test_set['corrected_search'] = test_set['corrected_search'].apply(lambda x : futher_preprocessing(x)) 

  cleaned_test_set2 = pd.DataFrame()
  cleaned_test_set2['title'] = test_set['cleaned_title'].apply(lambda x : futher_preprocessing_without_stem(x)) 
  cleaned_test_set2['brand'] = test_set['cleaned_brand'].apply(lambda x : futher_preprocessing_without_stem(x)) 
  cleaned_test_set2['description'] = test_set['cleaned_description'].apply(lambda x : futher_preprocessing_without_stem(x))
  cleaned_test_set2['attributes'] = test_set['cleaned_attributes'].apply(lambda x : futher_preprocessing_without_stem(x)) 
  cleaned_test_set2['search'] = test_set['cleaned_search'].apply(lambda x : futher_preprocessing_without_stem(x)) 
  cleaned_test_set2['corrected_search'] = test_set['corrected_search'].apply(lambda x : futher_preprocessing_without_stem(x)) 

  cleaned_test_set['brand'] = cleaned_test_set['brand'].replace(to_replace =[""], value ="missing_brand")
  cleaned_test_set2['brand'] = cleaned_test_set2['brand'].replace(to_replace =[""], value ="missing_brand")
  cleaned_test_set['search'] = cleaned_test_set['search'].replace(to_replace =[""], value ="missing_search")
  cleaned_test_set2['search'] = cleaned_test_set2['search'].replace(to_replace =[""], value ="missing_search")

  cleaned_test_set['attributes'] = cleaned_test_set['attributes'].apply(lambda x: re.sub('bullet \d\d ', '', x))
  cleaned_test_set2['attributes'] = cleaned_test_set2['attributes'].apply(lambda x: re.sub('bullet \d\d ', '', x))

  cleaned_test_set['description'] = cleaned_test_set['description'].apply(lambda x: re.sub('bullet \d\d ', '', x))
  cleaned_test_set2['description'] = cleaned_test_set2['description'].apply(lambda x: re.sub('bullet \d\d ', '', x))

  cleaned_test_set.rename(columns={"search": "raw_search"}, inplace=True)
  cleaned_test_set2.rename(columns={"search": "raw_search"}, inplace=True)

  #FEATURIZATION
  #set1
  data1 = cleaned_test_set.copy()

  data1['common_ST'] = common_words(data1,'corrected_search', 'title')
  data1['common_SD'] = common_words(data1,'corrected_search', 'description')
  data1['common_SB'] = common_words(data1,'corrected_search', 'brand')
  data1['common_r_ST'] = common_words(data1,'raw_search', 'title')
  data1['common_r_SD'] = common_words(data1,'raw_search', 'description')
  data1['common_r_SB'] = common_words(data1,'raw_search', 'brand')

  data1['num_common_ST'] = data1['common_ST'].apply(lambda x : len(x.split()))
  data1['num_common_SD'] = data1['common_SD'].apply(lambda x : len(x.split()))
  data1['num_common_SB'] = data1['common_SB'].apply(lambda x : len(x.split()))
  data1['num_common_r_ST'] = data1['common_r_ST'].apply(lambda x : len(x.split()))
  data1['num_common_r_SD'] = data1['common_r_SD'].apply(lambda x : len(x.split()))
  data1['num_common_r_SB'] = data1['common_r_SB'].apply(lambda x : len(x.split()))

  data1['cosine_ST'] = data1.apply(lambda row: cosine_similarity_sent(row['corrected_search'], row['title']), axis=1) 
  data1['cosine_SD'] = data1.apply(lambda row: cosine_similarity_sent(row['corrected_search'], row['description']), axis=1)
  data1['cosine_SB'] = data1.apply(lambda row: cosine_similarity_sent(row['corrected_search'], row['brand']), axis=1)
  data1['cosine_r_ST'] = data1.apply(lambda row: cosine_similarity_sent(row['raw_search'], row['title']), axis=1) 
  data1['cosine_r_SD'] = data1.apply(lambda row: cosine_similarity_sent(row['raw_search'], row['description']), axis=1)
  data1['cosine_r_SB'] = data1.apply(lambda row: cosine_similarity_sent(row['raw_search'], row['brand']), axis=1)

  data1['jacquard_ST'] = data1.apply(lambda row: jacquard_coefficient_sent(row['corrected_search'], row['title']), axis=1) 
  data1['jacquard_SD'] = data1.apply(lambda row: jacquard_coefficient_sent(row['corrected_search'], row['description']), axis=1)
  data1['jacquard_SB'] = data1.apply(lambda row: jacquard_coefficient_sent(row['corrected_search'], row['brand']), axis=1)
  data1['jacquard_r_ST'] = data1.apply(lambda row: jacquard_coefficient_sent(row['raw_search'], row['title']), axis=1) 
  data1['jacquard_r_SD'] = data1.apply(lambda row: jacquard_coefficient_sent(row['raw_search'], row['description']), axis=1)
  data1['jacquard_r_SB'] = data1.apply(lambda row: jacquard_coefficient_sent(row['raw_search'], row['brand']), axis=1)

  data1['len_description'] = data1['description'].apply(lambda x : len(x.split()))
  data1['len_brand'] = data1['brand'].apply(lambda x : len(x.split()))
  data1['len_title'] = data1['title'].apply(lambda x : len(x.split()))
  data1['len_search'] = data1['corrected_search'].apply(lambda x : len(x.split()))
  data1['len_r_search'] = data1['raw_search'].apply(lambda x : len(x.split()))

  data1['islast_ST'] = data1.apply(lambda row: row['corrected_search'].split()[-1] in row['title'].split(), axis=1)
  data1['islast_SD'] = data1.apply(lambda row: row['corrected_search'].split()[-1] in row['description'].split(), axis=1)
  data1['islast_SB'] = data1.apply(lambda row: row['corrected_search'].split()[-1] in row['brand'].split(), axis=1)
  data1['islast_r_ST'] = data1.apply(lambda row: row['raw_search'].split()[-1] in row['title'].split(), axis=1)
  data1['islast_r_SD'] = data1.apply(lambda row: row['raw_search'].split()[-1] in row['description'].split(), axis=1)
  data1['islast_r_SB'] = data1.apply(lambda row: row['raw_search'].split()[-1] in row['brand'].split(), axis=1)

  bool_cols = ['islast_ST', 'islast_SD', 'islast_SB', 'islast_r_ST', 'islast_r_SD', 'islast_r_SB']
  for col in bool_cols:
    data1[col] = data1[col].astype(int)

  #set2
  data2 = cleaned_test_set.copy()

  X_search = F2_tfidf_search.transform(data2['corrected_search'])
  truncated_search = F2_tsvd_search.transform(X_search)
  X_title = F2_tfidf_title.transform(data2['title'])
  truncated_title = F2_tsvd_title.transform(X_title)
  X_desc = F2_tfidf_desc.transform(data2['description'])
  truncated_desc = F2_tsvd_desc.transform(X_desc)
  trun_arr = np.hstack((truncated_search,truncated_title,truncated_desc))
  truncated_df = pd.DataFrame(trun_arr, index=cleaned_test_set.index)

  title_desc = data2["title"].astype(str) + ' ' + data2["description"].astype(str)
  X_title_desc = F2_tfidf_lsi.transform(title_desc)
  truncated_title_desc = F2_tsvd_lsi.transform(X_title_desc)
  X_search_ = F2_tfidf_lsi.transform(data2['corrected_search'])
  transformed_search = F2_tsvd_lsi.transform(X_search_)

  cos_sim = []
  for i in range(len(transformed_search)):
    cos_sim.append(cosine_similarity_vec(truncated_title_desc[i], transformed_search[i]))
  data2['lsi_cos_sim'] = cos_sim
  jaq_sim = []
  for i in range(len(transformed_search)):
    jaq_sim.append(jacquard_similarity_vec(truncated_title_desc[i], transformed_search[i]))
  data2['lsi_jaq_sim'] = jaq_sim
  inn_prod = []
  for i in range(len(transformed_search)):
    inn_prod.append(inner_product_vec(truncated_title_desc[i], transformed_search[i]))
  data2['lsi_inn_prod'] = inn_prod

  data2 = data2[['lsi_cos_sim', 'lsi_jaq_sim',	'lsi_inn_prod']]

  #set3
  data3 = cleaned_test_set.copy()

  data3['JM_ST'] = data3.apply(lambda row: lmir_jm_score(row['corrected_search'], row['title'], params_title_LM, 0.9), axis=1)
  data3['Dir_ST'] = data3.apply(lambda row: lmir_dir_score(row['corrected_search'], row['title'], params_title_LM, 12 ), axis=1)
  data3['AD_ST'] = data3.apply(lambda row: lmir_abs_score(row['corrected_search'], row['title'], 0.01 ), axis=1)
  data3['JM_SB'] = data3.apply(lambda row: lmir_jm_score(row['corrected_search'], row['brand'], params_brand_LM, 0.9), axis=1)
  data3['Dir_SB'] = data3.apply(lambda row: lmir_dir_score(row['corrected_search'], row['brand'], params_brand_LM, 1.5 ), axis=1)
  data3['AD_SB'] = data3.apply(lambda row: lmir_abs_score(row['corrected_search'], row['brand'], 0.01 ), axis=1)
  data3['JM_SD'] = data3.apply(lambda row: lmir_jm_score(row['corrected_search'], row['description'], params_desc_LM, 0.9), axis=1)
  data3['Dir_SD'] = data3.apply(lambda row: lmir_dir_score(row['corrected_search'], row['description'], params_desc_LM, 106 ), axis=1)
  data3['AD_SD'] = data3.apply(lambda row: lmir_abs_score(row['corrected_search'], row['description'], 0.01 ), axis=1)

  data3['bm25_ST'] = data3.apply(lambda row: bm25_score(row['corrected_search'], row['title'], params_title_bm25 ), axis=1)
  data3['bm25_SD'] = data3.apply(lambda row: bm25_score(row['corrected_search'], row['description'], params_desc_bm25 ), axis=1)
  data3['bm25_SB'] = data3.apply(lambda row: bm25_score(row['corrected_search'], row['brand'], params_brand_bm25 ), axis=1)


  dictionary = tfidf_w2v_params_search['dictionary']
  tfidf_words = tfidf_w2v_params_search['tfidf_words']
  search_tfidf_w2v_test = []; # the avg-w2v for each sentence/review is stored in this list
  for sentence in cleaned_test_set2['corrected_search']: # for each review/sentence
      vector = np.zeros(300) # as word vectors are of zero length
      tf_idf_weight =0; # num of words with a valid vector in the sentence/review
      for word in sentence.split(): # for each word in a review/sentence
          if (word in glove_words) and (word in tfidf_words):
              vec = model[word] # getting the vector for each word
              # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
              tf_idf = dictionary[word]*(sentence.count(word)) # getting the tfidf value for each word
              vector += (vec * tf_idf) # calculating tfidf weighted w2v
              tf_idf_weight += tf_idf
      if tf_idf_weight != 0:
          vector /= tf_idf_weight
      search_tfidf_w2v_test.append(vector)



  dictionary = tfidf_w2v_params_title['dictionary']
  tfidf_words = tfidf_w2v_params_title['tfidf_words']
  title_tfidf_w2v_test = []; # the avg-w2v for each sentence/review is stored in this list
  for sentence in cleaned_test_set2['title']: # for each review/sentence
      vector = np.zeros(300) # as word vectors are of zero length
      tf_idf_weight =0; # num of words with a valid vector in the sentence/review
      for word in sentence.split(): # for each word in a review/sentence
          if (word in glove_words) and (word in tfidf_words):
              vec = model[word] # getting the vector for each word
              # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
              tf_idf = dictionary[word]*(sentence.count(word)) # getting the tfidf value for each word
              vector += (vec * tf_idf) # calculating tfidf weighted w2v
              tf_idf_weight += tf_idf
      if tf_idf_weight != 0:
          vector /= tf_idf_weight
      title_tfidf_w2v_test.append(vector)


  dictionary = tfidf_w2v_params_desc['dictionary']
  tfidf_words = tfidf_w2v_params_desc['tfidf_words']
  desc_tfidf_w2v_test = []; # the avg-w2v for each sentence/review is stored in this list
  for sentence in cleaned_test_set2['description']: # for each review/sentence
      vector = np.zeros(300) # as word vectors are of zero length
      tf_idf_weight =0; # num of words with a valid vector in the sentence/review
      for word in sentence.split(): # for each word in a review/sentence
          if (word in glove_words) and (word in tfidf_words):
              vec = model[word] # getting the vector for each word
              # here we are multiplying idf value(dictionary[word]) and the tf value((sentence.count(word)/len(sentence.split())))
              tf_idf = dictionary[word]*(sentence.count(word)) # getting the tfidf value for each word
              vector += (vec * tf_idf) # calculating tfidf weighted w2v
              tf_idf_weight += tf_idf
      if tf_idf_weight != 0:
          vector /= tf_idf_weight
      desc_tfidf_w2v_test.append(vector)

  arr1 = np.array(search_tfidf_w2v_test)
  arr2 = np.array(title_tfidf_w2v_test)
  arr3 = np.array(desc_tfidf_w2v_test)
  tfidf_w2v_df = pd.DataFrame(np.hstack((arr1, arr2, arr3)), index=cleaned_test_set2.index)

  idf_dict = F3_SmM_params_title['idf_dict']
  N = F3_SmM_params_title['N']
  max_tf = []
  max_idf = []
  max_tfidf = []
  min_tf = []
  min_idf = []
  min_tfidf = []
  sum_tf = []
  sum_idf = []
  sum_tfidf = []
  for ind, row in cleaned_test_set.iterrows():
    search = row['corrected_search']
    text = row['title']
    tf_vals = []
    idf_vals = []
    tfidf_vals = []
    for word in search.split():
      if word in idf_dict.keys():
        tf = text.count(word)
        idf = idf_dict[word]
      else:
        tf = text.count(word)
        idf = np.log(N+1)
      tf_vals.append(tf)
      idf_vals.append(idf)
      tfidf_vals.append(tf*idf)
    max_tf.append(max(tf_vals))
    min_tf.append(min(tf_vals))
    sum_tf.append(sum(tf_vals))
    max_idf.append(max(idf_vals))
    min_idf.append(min(idf_vals))
    sum_idf.append(sum(idf_vals))
    max_tfidf.append(max(tfidf_vals))
    min_tfidf.append(min(tfidf_vals))
    sum_tfidf.append(sum(tfidf_vals))

  data3['max_tf_ST'] = max_tf
  data3['max_idf_ST'] = max_idf
  data3['max_tfidf_ST'] = max_tfidf
  data3['min_tf_ST'] = min_tf
  data3['min_idf_ST'] = min_idf
  data3['min_tfidf_ST'] = min_tfidf
  data3['sum_tf_ST'] = sum_tf
  data3['sum_idf_ST'] = sum_idf
  data3['sum_tfidf_ST'] = sum_tfidf

  idf_dict = F3_SmM_params_brand['idf_dict']
  N = F3_SmM_params_brand['N']
  max_tf = []
  max_idf = []
  max_tfidf = []
  min_tf = []
  min_idf = []
  min_tfidf = []
  sum_tf = []
  sum_idf = []
  sum_tfidf = []
  for ind, row in cleaned_test_set.iterrows():
    search = row['corrected_search']
    text = row['brand']
    tf_vals = []
    idf_vals = []
    tfidf_vals = []
    for word in search.split():
      if word in idf_dict.keys():
        tf = text.count(word)
        idf = idf_dict[word]
      else:
        tf = text.count(word)
        idf = np.log(N+1)
      tf_vals.append(tf)
      idf_vals.append(idf)
      tfidf_vals.append(tf*idf)
    max_tf.append(max(tf_vals))
    min_tf.append(min(tf_vals))
    sum_tf.append(sum(tf_vals))
    max_idf.append(max(idf_vals))
    min_idf.append(min(idf_vals))
    sum_idf.append(sum(idf_vals))
    max_tfidf.append(max(tfidf_vals))
    min_tfidf.append(min(tfidf_vals))
    sum_tfidf.append(sum(tfidf_vals))

  data3['max_tf_SB'] = max_tf
  data3['max_idf_SB'] = max_idf
  data3['max_tfidf_SB'] = max_tfidf
  data3['min_tf_SB'] = min_tf
  data3['min_idf_SB'] = min_idf
  data3['min_tfidf_SB'] = min_tfidf
  data3['sum_tf_SB'] = sum_tf
  data3['sum_idf_SB'] = sum_idf
  data3['sum_tfidf_SB'] = sum_tfidf

  idf_dict = F3_SmM_params_desc['idf_dict']
  N = F3_SmM_params_desc['N']
  max_tf = []
  max_idf = []
  max_tfidf = []
  min_tf = []
  min_idf = []
  min_tfidf = []
  sum_tf = []
  sum_idf = []
  sum_tfidf = []
  for ind, row in cleaned_test_set.iterrows():
    search = row['corrected_search']
    text = row['description']
    tf_vals = []
    idf_vals = []
    tfidf_vals = []
    for word in search.split():
      if word in idf_dict.keys():
        tf = text.count(word)
        idf = idf_dict[word]
      else:
        tf = text.count(word)
        idf = np.log(N+1)
      tf_vals.append(tf)
      idf_vals.append(idf)
      tfidf_vals.append(tf*idf)
    max_tf.append(max(tf_vals))
    min_tf.append(min(tf_vals))
    sum_tf.append(sum(tf_vals))
    max_idf.append(max(idf_vals))
    min_idf.append(min(idf_vals))
    sum_idf.append(sum(idf_vals))
    max_tfidf.append(max(tfidf_vals))
    min_tfidf.append(min(tfidf_vals))
    sum_tfidf.append(sum(tfidf_vals))

  data3['max_tf_SD'] = max_tf
  data3['max_idf_SD'] = max_idf
  data3['max_tfidf_SD'] = max_tfidf
  data3['min_tf_SD'] = min_tf
  data3['min_idf_SD'] = min_idf
  data3['min_tfidf_SD'] = min_tfidf
  data3['sum_tf_SD'] = sum_tf
  data3['sum_idf_SD'] = sum_idf
  data3['sum_tfidf_SD'] = sum_tfidf

  data3 = data3.iloc[:,6:]

  #MODELLING
  X1 = pd.concat([data1[feat_set1_comb], data2, data3], axis=1)
  pred_xgb = F1_xgb.predict(X1) 
  pred_rf = F1_rf.predict(X1) 
  pred_ridge = F1_ridge.predict(F1_scaler_ridge.transform(X1)) 
  pred_lasso = F1_lasso.predict(F1_scaler_lasso.transform(X1)) 
  pred_en = F1_en.predict(F1_scaler_en.transform(X1)) 
  pred_svr = F1_svr.predict(F1_scaler_svr.transform(X1)) 
  pred_dt = F1_dt.predict(X1) 
  arr = np.hstack((pred_xgb.reshape(-1,1),
                  pred_rf.reshape(-1,1), 
                  pred_dt.reshape(-1,1), 
                  pred_svr.reshape(-1,1),
                  pred_ridge.reshape(-1,1),
                  pred_lasso.reshape(-1,1), 
                  pred_en.reshape(-1,1)))
  F1_df = pd.DataFrame(arr, columns=['f1_xgb', 'f1_rf', 'f1_dt', 'f1_svr', 'f1_ridge', 'f1_lasso', 'f1_en'], index=X1.index)

  X2 = pd.concat([data1[feat_set1_comb], data2, data3, tfidf_w2v_df], axis=1)
  pred_svr = F2_svr.predict(F2_scaler_svr.transform(X2)) 
  pred_rf = F2_rf.predict(X2) 
  pred_ridge = F2_ridge.predict(F2_scaler_ridge.transform(X2)) 
  pred_lasso = F2_lasso.predict(F2_scaler_lasso.transform(X2)) 
  pred_en = F2_en.predict(F2_scaler_en.transform(X2)) 
  pred_dt = F2_dt.predict(X2) 
  arr = np.hstack((pred_svr.reshape(-1,1),
                  pred_rf.reshape(-1,1),                                    
                  pred_ridge.reshape(-1,1),
                  pred_lasso.reshape(-1,1), 
                  pred_en.reshape(-1,1),
                  pred_dt.reshape(-1,1),))
  F2_df = pd.DataFrame(arr, columns=['f2_svr', 'f2_rf', 'f2_ridge', 'f2_lasso', 'f2_en', 'f2_dt'], index=X2.index)

  X3 = pd.concat([data1[feat_set1_comb], data2, data3, truncated_df], axis=1)
  pred_dt = F3_dt.predict(X3) 
  pred_ridge = F3_ridge.predict(F3_scaler_ridge.transform(X3)) 
  pred_lasso = F3_lasso.predict(F3_scaler_lasso.transform(X3)) 
  pred_en = F3_en.predict(F3_scaler_en.transform(X3)) 
  arr = np.hstack((pred_dt.reshape(-1,1), 
                  pred_ridge.reshape(-1,1),
                  pred_lasso.reshape(-1,1), 
                  pred_en.reshape(-1,1)))
  F3_df = pd.DataFrame(arr, columns=['f3_dt', 'f3_ridge', 'f3_lasso', 'f3_en'], index=X3.index)

  #FINAL
  test_x = pd.concat([F1_df, F2_df, F3_df], axis=1)
  test_x_std = final_scaler.transform(test_x)
  test_y_pred = final_ridge.predict(test_x_std)

  return test_y_pred

In [ ]:
def get_candidates(search, N):
  search = preprocessing_search(search)
  search = corrected_term(search)
  tokenized_query = search.split(" ")
  candidates = bm25_model.get_top_n(tokenized_query, product_text, n=N)
  candidate_products = database[database['text'].isin(candidates)].drop('text', axis=1)
  candidate_products['search_term'] = search
  reorder_cols = ['product_uid', 'product_title', 'search_term', 'combined_attr', 'brand', 'product_description']
  return candidate_products[reorder_cols]

In [ ]:
def main(srch, n):
  test_set = get_candidates(srch, 100)
  test_set['relevance'] = get_search_relevance(test_set)
  return test_set.sort_values('relevance', ascending=False).iloc[:n]['product_title']

## Final Testing

In [ ]:
import time 
start = time.time()

search = 'pan'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

end = time.time()
print('Time taken:', round(end-start,3), 'seconds')

The most relevant products to the search "pan" are:
1 Ozeri 8 in., 10 in., 12 in. Green Earth Frying Pan Set with Textured Ceramic Non-Stick Coating (100% PTFE and PFOA Free)
2 Range Kleen 10 in. Preferred Nonstick Fry Pan in Stainless Steel
3 Rubbermaid 9.38 in. L x 10 in. W x 11.75 in. H Metal in Cabinet Stand Alone Pan Organizer
4 Lava Signature 10-1/2 in. x 18-1/2 in. Enameled Cast Iron Roasting-Baking Pan in Orange Spice
5 Lava ECO 10-1/2 in. x 15-1/2 in. Enameled Cast Iron Grill Pan in Slate Black
6 Lava Signature 10-1/2 in. x 18-1/2 in. Enameled Cast Iron Roasting-Baking Pan in Cayenne Red
7 Lava ECO 10-1/2 in. x 19-3/4 in. Enameled Cast Iron Reversible Grill and Griddle Pan in Slate Black
8 Estwing 20 oz. 14 in. Steel Gold Pan
9 Lava ECO 15 in. x 13-1/4 in. Enameled Cast Iron Round Grill Pan in Slate Black
10 Lava ECO 10-1/2 in. x 13-1/2 in. Enameled Cast Iron Square Grill Pan in Slate Black
Time taken: 5.14 seconds


In [ ]:
search = 'air conditioner'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "air conditioner" are:
1 Danby 10,000 BTU Window Air Conditioner with Remote
2 Danby 15,000 BTU Window Air Conditioner with Remote
3 LG Electronics 7,000 BTU Window Air Conditioner with Cool, Heat and Remote
4 LG Electronics 7,000 BTU Window Air Conditioner with Cool, Heat and Remote
5 Danby 12,000 BTU Window Air Conditioner with Remote
6 Whynter 14,000 BTU Portable Air Conditioner with Dehumidifer and Remote
7 LG Electronics 12,000 BTU Window Air Conditioner with Cool, Heat and Remote
8 Honeywell 14,000 BTU Portable Air Conditioner with Remote Control in Black and Silver
9 LG Electronics 23,500 BTU Window Air Conditioner with Cool, Heat and Remote
10 LG Electronics 7,500 BTU 115-Volt Window Air Conditioner with Cool, Heat and Remote


In [ ]:
search = 'aor condiotioner'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "aor condiotioner" are:
1 Danby 10,000 BTU Window Air Conditioner with Remote
2 Danby 15,000 BTU Window Air Conditioner with Remote
3 LG Electronics 7,000 BTU Window Air Conditioner with Cool, Heat and Remote
4 LG Electronics 7,000 BTU Window Air Conditioner with Cool, Heat and Remote
5 Danby 12,000 BTU Window Air Conditioner with Remote
6 Whynter 14,000 BTU Portable Air Conditioner with Dehumidifer and Remote
7 LG Electronics 12,000 BTU Window Air Conditioner with Cool, Heat and Remote
8 Honeywell 14,000 BTU Portable Air Conditioner with Remote Control in Black and Silver
9 LG Electronics 23,500 BTU Window Air Conditioner with Cool, Heat and Remote
10 LG Electronics 7,500 BTU 115-Volt Window Air Conditioner with Cool, Heat and Remote


In [ ]:
search = 'warm water'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "warm water" are:
1 EcoSmart Energy Smart 200-Light LED Warm White Icicle Light Set
2 AquaPower 120-Volt 3 kW 0.46 GPM Compact Point-of-Use Electric Tankless Water Heater
3 AquaPower 240-Volt 5.7 kW 0.87 GPM Compact Point-of-Use Electric Tankless Water Heater
4 AquaPower 120-Volt 1.8 kW 0.27 GPM Compact Point-of-Use Electric Tankless Water Heater
5 60-Light Outdoor Warm White Decorative LED String Light
6 Zojirushi 4-Liter Micom Water Boiler and Warmer
7 Zojirushi 3-Liter Micom Water Boiler and Warmer
8 Pure Guardian 1.5 gal. 100-Hour Warm and Cool Mist Ultrasonic Humidifier
9 Crane 1 Gal. Warm Mist Humidifier - Aqua
10 Honeywell 1 Gal. Warm Moisture Filter Free Humidifier


In [ ]:
search = 'cut hair'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "cut hair" are:
1 Remington Power Beard Hair Trimmer
2 Andis 1600-Watt Hang-Up Hair Dryer with Light
3 Fun World Grave Breaker with Hair and Lite-Up Eyes
4 Original FlexiSnake Hair Clog Tool
5 DANCO Hair Catcher for Shower in White
6 Andis 1600-Watt Hang-Up Ionic Hair Dryer with Light
7 DANCO Hair Catcher for Shower Drain in Chrome
8 interDesign Classico Over Cabinet Hair Care Organizer in Chrome
9 Andis 1600-Watt Hang-Up Ionic Hair Dryer with Light
10 Instant Power 67.6 oz. Hair and Grease Drain Opener


In [ ]:
search = 'orange'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "orange" are:
1 Orange GLO 32 oz. 4-in-1 Hardwood Floor Cleaner and Polish
2 Orange 3-Tier Utility Cart
3 Orange GLO 32 oz. Hardwood Floor Cleaner
4 Stanley Panel Carry - Orange
5 Brite Star LED Orange Battery Operated Pumpkin Lights (Set of 10)
6 The Hillman Group 48 in. Reflective Rod Orange
7 Orange GLO 32 oz. Wood Furniture Cleaner and Polish
8 ArcMate 36 in. Industrial Orange Pick Up Reacher Tool
9 Everbilt 16 in. Orange Ground Stake
10 Everbilt 11 in. Orange Ground Stake


In [ ]:
search = 'foot'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "foot" are:
1 Johnson 13 ft. Aluminum Grade Rod
2 Backyard Discovery Installed 10 ft. x 10 ft. Cedar Pergola
3 Backyard Discovery 10 ft. x 12 ft. Cedar Pergola
4 Lufkin 6 ft. Folding Wood Rule
5 King Canopy 12 ft. W x 20 ft. D Steel Expandable Canopy
6 Arrow Sheridan 10 ft. x 8 ft. Steel Storage Shed
7 Ceilingmax 100 sq. ft. Ceiling Grid Kit White
8 Sigman 10 ft. x 20 ft. Silver Heavy Duty Tarp
9 Pacific Casual Hampton Bay 10 ft. x 10 ft. Pitched Roof Line Portable Patio Gazebo Replacement Canopy
10 PRO-SERIES 12 ft. x 7 ft. x 5 ft. 2-Story Commercial Grade Rolling Scaffold Tower 1,500 lb. Load Capacity


In [ ]:
search = '10'
print('The most relevant products to the search "{}" are:'.format(search))
for i, prod in enumerate(main(search, 10)):
  print(i+1, prod)

The most relevant products to the search "10" are:
1 10 ft. x 10 ft. 3-Tier Gazebo
2 Lumabase Tan/Kraft Electric Luminaria Kit (Set of 10)
3 interDesign Una 10 in. Waste Can in Black
4 interDesign Una 10 in. Waste Can in Slate Gray
5 Lumabase Pathway Clear String Lights (Set of 10)
6 10 ft. x 10 ft. Vinyl-Coated Steel Carport
7 Razor-Back 10 in. x 10 in. Steel Tamper
8 TruAire 10 in. x 10 in. 3-Way Wall/Ceiling Register
9 TruAire 10 in. x 10 in. White Return Air Grille
10 10 in. x 10 in. x 8 in. Concrete Deck Block


Run Time

In [ ]:
import time 
search = 'pan'

time_taken = []

for i in range(100):
  start = time.time()
  main(search, 10)
  end = time.time()
  time_taken.append(end-start)
  
print('Average Run Time: {}'.format(np.mean(time_taken)))

Average Run Time: 5.092467794418335


In [ ]:
import time 
search = 'air conditioner black whirpool'

time_taken = []

for i in range(10):
  start = time.time()
  main(search, 10)
  end = time.time()
  time_taken.append(end-start)
  
print('Average Run Time: {}'.format(np.mean(time_taken)))

Average Run Time: 5.598297357559204
